### Retrival Augmentented Generation
Retrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response. Large Language Models (LLMs) are trained on vast volumes of data and use billions of parameters to generate original output for tasks like answering questions, translating languages, and completing sentences. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, all without the need to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts.

#### Vector embeddings
Let's begin with the fundamental building blocks of how retrieval works

In [ ]:
%pip install langchain_cohere -q
%pip install spacy -q
#ignore error

In [ ]:
# now you need to run this in a terminal window
# python -m spacy download en_core_web_md
# now restart your kernel

Standard imports for the libraires we will be using in this notebook.  Try to keep your imports in the first cell so this can this code can more easliy be converted into a python program later

In [ ]:
import boto3
import pandas as pd
import json
import numpy as np
import pyarrow
from langchain.embeddings import BedrockEmbeddings
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_cohere import CohereEmbeddings

# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

#### Lets define functions that will use various embedding models so we can generate vector embeddings
Spacey

In [ ]:
def generate_spacy_vector_embedding(text):
    embedder = SpacyEmbeddings(model_name="en_core_web_md")
    query_embedding = embedder.embed_query(text)

    return(np.array(query_embedding))

Huggingface Bert

In [ ]:
def generate_huggingface_vector_embedding(text):
    embeddings = HuggingFaceEmbeddings(model_name="bert-base-uncased")
    embedded_texts = embeddings.embed_query(text_data)
    return(embedded_texts)

Cohere

In [ ]:
# send in an array size of one and only return the 0th element
def generate_cohere_vector_embedding(text_data):
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    
    # Create the JSON payload for the request
    json_params = {
            'texts': [text_data],
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(np.array(response['embeddings'][0]))


Amazon Titan

In [ ]:
# Let's generate a dense vector using Amazon Titan with LangChain
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))



This is the mathmatical formula to calcuate cosine similarity between 2 vectors

In [ ]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



#### Persistance of embeddings for later retrieval
 In order to do semantic search and retrieve relevant content we need to store that content for later use.  We can store the embedding in several different persistence technologies.  To start simply let's store the data in memory using pandas dataframe.

In [ ]:
def clean_value(value):
    value_str = str(value)
    cleaned_value = ''.join(char for char in value_str if char.isalnum() or char.isspace())
    return cleaned_value

In [ ]:
def limit_string_size(x, max_chars=2048):
    # Check if the input is a string
    if isinstance(x, str):
        return x[:max_chars]
    else:
        return x

In [ ]:
# clean abstract text
df = pd.read_csv('INSERT YOUR COOL DATASET HERE')
df['abstract'] = df['abstract'].apply(clean_value)
# for cohere we need to limit the string size to 2048
#df['abstract'] = df['abstract'].apply(limit_string_size)
df

In [ ]:
# Generate embeddings using spaCy
# Add a new column 'embedded_abstract' by applying the function to an existing column
# This step takes a while so I did it for you and saved the output as pickle
#df['embedded_abstract'] = df['abstract'].apply(generate_spacy_vector_embedding)


### Retrieval from embedded sources
Now that we have a dataframe with embedded content of interest, we can use semantic similarity to retrieve the right data to feed to an LLM

In [ ]:
# Let's setup a query that a used might ask
query = "What is the latest research for broken ribs in children"

#### Titan Embeddings

In [ ]:
dft = pd.read_pickle('data/embedded_df.pkl')
# Let's search our records for a good semantic search
query_vector = generate_titan_vector_embedding(query)

results = []
# Iterate over each row in the DataFrame
for index, row in dft.iterrows():
    # Extract the value from the specified column
    article_embedding = row['embedded_abstract']
    results.append((index, cosine_similarity(article_embedding, query_vector)))
    #print (index, value)

results.sort(key=lambda x: x[1], reverse=True)
i = 0
# Print the sorted data
print("Here are a few articles that may match your interest:")
for item in results:
    article_title = dft.iloc[item[0]]['title']
    print(f"Abtract: '{article_title}' with a cosine match of: {item[1]}")
    i=i+1
    if i == 5:
        break

### Assignment
Using a dataset of your choosing try and retrieve relevant context for given user input.


1) Use kaggle or other source of data that has information that you want to use for context to an LLM call.

Upload your dataset to the data folder and briefly describe why you selected this data

In [4]:
# Your response here

2. Data ingestion -> In reviewing your datasource what needs to be done?  Is there data cleansing involved is there chunking required? Why or why not?

Perform and data cleansinfg or chunking here

In [5]:
# Your code here

3. Load the data into a dataframe and embed the data for retrieval.

Which embedding model do you pick and why?

In [6]:
# Your code here

4. Now test your retrieval by using a few test queries to sample the data.

How did you determine your retrieval threshold (i.e. What cosine similarty did you pick, did it remain consistent for different queries?

In [7]:
# Your code here